In [ ]:
cSet = 300
cMar = 100
cPen = 1600
R = {} #裝各期的recursive，即該期最佳cost
S = {} #裝各期各x下的value
K = {} #裝各期最佳key值
X = 6 # 商品最大數量
T = 3 #總期數
for t in range(T, 0, -1): # 建立字典中的字典，每個小字典裝當期值
    S[t] = {}
    print(S)
for t in range(T, 0, -1): # 執行backward induction
    for x in range(0, X+1): # 所有的生產量都嘗試看看
        if t == T: # 如果現在是最後一期
            if x == 0 :
                reward = 0
            else:
                reward = cSet+x*cMar
            value = reward +((1/2)**x) * 1600
        else: #否則
            if x == 0:
                reward = 0
            else:
                reward = cSet+x*cMar
            value = reward+((1/2)**x)*R[t+1]
        S[t][t,x] = value  #紀錄當下cost

    minval = min(S[t].values()) #找出當期最小值
    R[t] = minval#當期最小的cost

    for v in S[t]:
        if S[t][v] == minval:
            if t not in K:
                K[t] = [v[1]] #第t期的最佳解是X=x
            else:
                K[t].append(v[1])  
print(K)
print('----'*10)
print(R)
print('----'*10)
for i in S:
    print(S[i])

{3: {}}
{3: {}, 2: {}}
{3: {}, 2: {}, 1: {}}
{3: [3, 4], 2: [2, 3], 1: [2]}
----------------------------------------
{3: 800.0, 2: 700.0, 1: 675.0}
----------------------------------------
{(3, 0): 1600.0, (3, 1): 1200.0, (3, 2): 900.0, (3, 3): 800.0, (3, 4): 800.0, (3, 5): 850.0, (3, 6): 925.0}
{(2, 0): 800.0, (2, 1): 800.0, (2, 2): 700.0, (2, 3): 700.0, (2, 4): 750.0, (2, 5): 825.0, (2, 6): 912.5}
{(1, 0): 700.0, (1, 1): 750.0, (1, 2): 675.0, (1, 3): 687.5, (1, 4): 743.75, (1, 5): 821.875, (1, 6): 910.9375}


In [ ]:
import numpy as np  #最後四捨五入需要用到的套件

init_money = 3  #起始金額
T = 3  #總期數

max_money_on_hand = [0]
for i in range(0,T):
    max_money_on_hand.append(init_money * (2**(i)))
#print(max_money_on_hand)

O = {}  #裝各期各初始金額下最佳決策
money_prob = {} #裝各期各金額各決策下的解 (未含必輸必贏以及最後一期)

for t in range(T,0,-1):
    max_money = max_money_on_hand[t]
    O[t] = {}  #重要，在O的字典裡插入字典，每一期為一個小字典
    
    if t == T: #最後一期
        for money in range(0, max_money+1):
            if 2*money < 5:  #all in 還必輸
                O[t][money] = (0, [0]) #(最佳機率、最佳決策)
            elif money >= 5:  #穩贏
                best_action = []
                for i in range(0, money - 5 + 1): #不能出完錢口袋剩不到五塊
                    best_action.append(i)
                O[t][money] = (1, best_action)
            else:
                best_action = []
                for i in range(5-money, money + 1): #如果贏了至少贏後要有五塊
                    best_action.append(i)
                O[t][money] = (2/3, best_action)
        #print(O[3])

    elif t != 1:  #不會包含最後一期因為if elif的順序
        
        money_prob[t] = {}
        for money in range(0, max_money+1):
            if money * (2**(T-t+1)) < 5: #二的剩餘期數加一次方  #每期all in但最終必輸
                O[t][money] = (0, [0])
                
            elif money >= 5:  #穩贏
                best_action = []
                for i in range(0, money - 5 + 1):
                    best_action.append(i)
                O[t][money] = (1, best_action)
                
            else:  #其他
                money_prob[t][money] = {}
                max_prob = -1 #最小值亂設比零小就好
                for i in range(0 , money + 1):
                    temp_prob = 1/3 * O[t+1][money-i][0] + 2/3 * O[t+1][money+i][0] #記得補0
                    money_prob[t][money][i] = temp_prob #第t期手上有money元做了i決策             
                    
                    if temp_prob > max_prob:
                        max_prob = temp_prob
                        best_action = [i]
                        
                    elif temp_prob == max_prob:
                        best_action.append(i)  

                    #print(max_prob, best_action)
                #print(money_prob)
                
                O[t][money] = (max_prob, best_action) #第t期手上有money元最佳機率為maxprob最佳決策為bestaction
        #print(O[2])
        
    elif t == 1: #第1期
        money_prob[1] = {} #在money_prob字典中又建了第一期(key值為1)的小字典
        money_prob[1][3] = {} #在第一期的小字典中又建了手上有三塊(key值為3)的小字典
        max_prob = -1
        
        for i in range(0, max_money+1): #0, 1, 2, 3
            temp_prob = 1/3 * O[t+1][3-i][0] + 2/3 * O[t+1][3+i][0]
            money_prob[1][3][i] = temp_prob
            
            if temp_prob > max_prob:
                max_prob = temp_prob
                best_action = [i]
            elif temp_prob == max_prob:
                best_action.append(i)
        
        O[t][3] =(max_prob, best_action)
        #print(O[1])

print(money_prob)
print()
print(O)
print()
for i in money_prob:
    print('期數',i)
    for j in money_prob[i]:
        print('持有金額',j)
        for k in money_prob[i][j]:
            print('付出金額',k,'獲勝機率:',money_prob[i][j][k])
print()
for i in O:
    print("期數", i)
    for j in O[i]:
        print("持有金額", j, "獲勝機率", np.round(O[i][j][0],2), "最佳決策", O[i][j][1])


{2: {2: {0: 0.0, 1: 0.4444444444444444, 2: 0.4444444444444444}, 3: {0: 0.6666666666666666, 1: 0.4444444444444444, 2: 0.6666666666666666, 3: 0.6666666666666666}, 4: {0: 0.6666666666666666, 1: 0.8888888888888888, 2: 0.6666666666666666, 3: 0.6666666666666666, 4: 0.6666666666666666}}, 1: {3: {0: 0.6666666666666666, 1: 0.7407407407407407, 2: 0.6666666666666666, 3: 0.6666666666666666}}}

{3: {0: (0, [0]), 1: (0, [0]), 2: (0, [0]), 3: (0.6666666666666666, [2, 3]), 4: (0.6666666666666666, [1, 2, 3, 4]), 5: (1, [0]), 6: (1, [0, 1]), 7: (1, [0, 1, 2]), 8: (1, [0, 1, 2, 3]), 9: (1, [0, 1, 2, 3, 4]), 10: (1, [0, 1, 2, 3, 4, 5]), 11: (1, [0, 1, 2, 3, 4, 5, 6]), 12: (1, [0, 1, 2, 3, 4, 5, 6, 7])}, 2: {0: (0, [0]), 1: (0, [0]), 2: (0.4444444444444444, [1, 2]), 3: (0.6666666666666666, [0, 2, 3]), 4: (0.8888888888888888, [1]), 5: (1, [0]), 6: (1, [0, 1])}, 1: {3: (0.7407407407407407, [1])}}

期數 2
持有金額 2
付出金額 0 獲勝機率: 0.0
付出金額 1 獲勝機率: 0.4444444444444444
付出金額 2 獲勝機率: 0.4444444444444444
持有金額 3
付出金額 0 獲勝機率: